In [1]:
from pprint import pprint
import requests
import time
import pandas as pd

# Function to use requests.post to make an API call to the subgraph URL
def run_query(query):
    # Endpoint where you are making the request
    request = requests.post('https://gateway-arbitrum.network.thegraph.com/api/829116bfdd9d51f6394344cac20289b0/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. Return code is {}. {}'.format(request.status_code, query))

# Function to switch tokens if token0 has more transactions
def switch_token(pair):
    if int(pair['token0']['txCount']) > int(pair['token1']['txCount']):
        pair['reserve00'], pair['reserve11'] = pair['reserve1'], pair['reserve0']
        pair['token00'], pair['token11'] = pair['token1'], pair['token0']
    else:
        pair['reserve00'], pair['reserve11'] = pair['reserve0'], pair['reserve1']
        pair['token00'], pair['token11'] = pair['token0'], pair['token1']

# Function to get pair info for a given token address with token1 always being WETH
def get_pair_info(token_address):
    query_template = '''
    {
      pairs(where: {token0: "token_address", token1: "WETH_ADDRESS"}) {
        id
        token0{
          id
          symbol
          name
          txCount
          totalLiquidity
        }
        token1{
          id
          symbol
          name
          txCount
          totalLiquidity
        }
        reserve0
        reserve1
        totalSupply
        reserveUSD
        reserveETH
        txCount
        createdAtTimestamp
        createdAtBlockNumber
      }
    }
    '''
    WETH_ADDRESS = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"  # WETH address on Arbitrum
    query = query_template.replace('token_address', token_address).replace('WETH_ADDRESS', WETH_ADDRESS)
    print("Executing query:")
    print(query)  # Print the query for debugging
    result = run_query(query)
    
    if 'data' in result and 'pairs' in result['data'] and len(result['data']['pairs']) > 0:
        pair = result['data']['pairs'][0]  # Assuming we want the first matching pair
        switch_token(pair)
        year = time.gmtime(int(pair['createdAtTimestamp'])).tm_year
        month = time.gmtime(int(pair['createdAtTimestamp'])).tm_mon
        day = time.gmtime(int(pair['createdAtTimestamp'])).tm_mday
        pair['createdAtTimestamp'] = f"{year}-{month}-{day}"
        return pair
    else:
        print("No data found for the given token address with WETH.")
        pprint(result)  # Print the full result for debugging
        return None

# Main code to get user input for the token address
token_address = "0x8115def7d6ab9ab41ef618febcccf26ee9a54e8b"  # Replace with the actual token address
pair_info = get_pair_info(token_address)
if pair_info:
    pprint(pair_info)
    # Convert the pair_info to a DataFrame
    df = pd.json_normalize(pair_info)
    # Save the DataFrame to a CSV file
    df.to_csv('input.csv', index=False)
    print("Data has been saved to input.csv")
else:
    print("No data found for the given token address with WETH.")


Executing query:

    {
      pairs(where: {token0: "0x8115def7d6ab9ab41ef618febcccf26ee9a54e8b", token1: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}) {
        id
        token0{
          id
          symbol
          name
          txCount
          totalLiquidity
        }
        token1{
          id
          symbol
          name
          txCount
          totalLiquidity
        }
        reserve0
        reserve1
        totalSupply
        reserveUSD
        reserveETH
        txCount
        createdAtTimestamp
        createdAtBlockNumber
      }
    }
    
{'createdAtBlockNumber': '20011236',
 'createdAtTimestamp': '2024-6-3',
 'id': '0xa1057899dbe0890bbe6b4b6836fef4cc1fdac0b3',
 'reserve0': '0.000000161',
 'reserve00': '0.000000161',
 'reserve1': '0.000000000000006256',
 'reserve11': '0.000000000000006256',
 'reserveETH': '0.000000000000012512',
 'reserveUSD': '0.00000000004735020883907936726601258836147742',
 'token0': {'id': '0x8115def7d6ab9ab41ef618febcccf26ee9a54e8b'

In [2]:
df = pd.read_csv('input.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   createdAtBlockNumber    1 non-null      int64  
 1   createdAtTimestamp      1 non-null      object 
 2   id                      1 non-null      object 
 3   reserve0                1 non-null      float64
 4   reserve1                1 non-null      float64
 5   reserveETH              1 non-null      float64
 6   reserveUSD              1 non-null      float64
 7   totalSupply             1 non-null      int64  
 8   txCount                 1 non-null      int64  
 9   reserve00               1 non-null      float64
 10  reserve11               1 non-null      float64
 11  token0.id               1 non-null      object 
 12  token0.name             1 non-null      object 
 13  token0.symbol           1 non-null      object 
 14  token0.totalLiquidity   1 non-null      float6

In [3]:
df.head()

,createdAtBlockNumber,createdAtTimestamp,id,reserve0,reserve1,reserveETH,reserveUSD,totalSupply,txCount,reserve00,...,token00.id,token00.name,token00.symbol,token00.totalLiquidity,token00.txCount,token11.id,token11.name,token11.symbol,token11.totalLiquidity,token11.txCount
0,20011236,2024-6-3,0xa1057899dbe0890bbe6b4b6836fef4cc1fdac0b3,1.610000e-07,6.200000e-15,1.250000e-14,4.735020e-11,0,33,1.610000e-07,...,0x8115def7d6ab9ab41ef618febcccf26ee9a54e8b,PEPEWIF,PEPEWIF,1.610000e-07,33,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,292361.382058,169796117


In [13]:
import requests
import json
from bs4 import BeautifulSoup
import re

mint_query_template = '''
{
  mints(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt: "%s"  }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''

mint_query_first = '''
{
  mints(first: 1, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''


swap_query_template = '''
{
  swaps(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s }) {
      amount0In
      amount0Out
      amount1In
      amount1Out
      to
      sender
      timestamp
 }
}
'''


burn_query_template = '''
{
  burns(first: 1000, orderBy: timestamp, orderDirection: asc, where:{ pair: "%s" , timestamp_gt:%s }) {
      amount0
      amount1
      to
      sender
      timestamp
 }
}
'''


def run_query(query):

    # endpoint where you are making the request
    request = requests.post('https://gateway-arbitrum.network.thegraph.com/api/829116bfdd9d51f6394344cac20289b0/subgraphs/id/A3Np3RQbaBA6oKJgiwDJeo5T3zrYfGHPWFYayMwtNDum',
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

def call_theGraph_mint(pair_id):
    pair_id = "0xa1057899dbe0890bbe6b4b6836fef4cc1fdac0b3"
    mint_array = []
    timestamp = 0
    try:
      while(True):
        query = mint_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['mints']) < 1000):
          mint_array.extend(result['data']['mints'])
          break

        mint_array.extend(result['data']['mints'])
        timestamp = result['data']['mints'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_swap')
      print(e)

    return mint_array

def call_theGraph_swap(pair_id):
    swap_array = []
    timestamp = 0
    try:
      while(True):
        query = swap_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['swaps']) < 1000):
          swap_array.extend(result['data']['swaps'])
          break

        swap_array.extend(result['data']['swaps'])
        timestamp = result['data']['swaps'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_swap')
      print(e)

    return swap_array

def call_theGraph_burn(pair_id):
    burn_array = []
    timestamp = 0
    try:
      while(True):
        query = burn_query_template % (pair_id,timestamp)
        result = run_query(query)

        if(len(result['data']['burns']) < 1000):
          burn_array.extend(result['data']['burns'])
          break

        burn_array.extend(result['data']['burns'])
        timestamp = result['data']['burns'][999]['timestamp']
    except Exception as e:
      print('error in theGraph_burn')
      print(e)

    return burn_array





In [14]:
datas = pd.read_csv('input.csv').to_dict('records')

In [15]:
for data in datas:
    pair_address = data['id']
    token_address = data['token00.id']

In [16]:
mint_data_transaction = call_theGraph_mint(pair_address)
swap_data_transaction = call_theGraph_swap(pair_address)
burn_data_transaction = call_theGraph_burn(pair_address)

In [17]:
def get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction):
  #mint_data_transaction
  swap_len = len(swap_data_transaction)
  burn_len = len(burn_data_transaction)
  #Case 1 Swap / Burn
  if(swap_len == 0 and burn_len == 0):
    return int(mint_data_transaction[-1]['timestamp'])
  #Case 2 Swap_transaction
  if(swap_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp']))
  #Case 3 Burn Transaction
  if(burn_len == 0):
    return int(max(mint_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))
  #Case 4
  return int(max(mint_data_transaction[-1]['timestamp'],burn_data_transaction[-1]['timestamp'],swap_data_transaction[-1]['timestamp']))


In [18]:
initial_timestamp = int(mint_data_transaction[0]['timestamp'])
last_timestamp = get_last_timestamp(mint_data_transaction,swap_data_transaction,burn_data_transaction)
active_period = last_timestamp - initial_timestamp  # first feature of model

In [19]:
swap_count = len(swap_data_transaction) #second feature of model

In [20]:
def get_swap_mean_period(swap_data_transaction,initial_timestamp):
    count = len(swap_data_transaction)
    if(count == 0):
      return 0
    swap_time_add = 0
    for transaction in swap_data_transaction:
      swap_time_add = swap_time_add +  int(transaction['timestamp']) - initial_timestamp
    return swap_time_add / count

def get_burn_mean_period(burn_data_transaction,initial_timestamp):
    count = len(burn_data_transaction)
    if(count == 0):
      return 0
    burn_time_add = 0
    for transaction in burn_data_transaction:
      burn_time_add = burn_time_add + int(transaction['timestamp']) - initial_timestamp
    return burn_time_add / count

In [21]:
burn_mean_period = int(get_burn_mean_period(burn_data_transaction,initial_timestamp))   #third feataure
swap_mean_period = int(get_swap_mean_period(swap_data_transaction,initial_timestamp))   #fourth feature

In [22]:
def token_index(data):
    if(data['token0.name'] == 'Wrapped Ether'):
        return 1
    else:
        return 0
    
def swap_IO_rate(swap_data_transaction,index):
  swapIn = 0
  swapOut = 0
  if(index == 1):
    for data in swap_data_transaction:
      if(data['amount0In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn + 1
  else:
    for data in swap_data_transaction:
      if(data['amount1In'] == '0'):
        swapOut = swapOut + 1
      else:
        swapIn = swapIn +1

  return swapIn,swapOut

In [23]:
swapIn,swapOut = swap_IO_rate(swap_data_transaction,token_index(data))  #fifth and sixth feature

In [24]:
def get_holders(token_id):
    repos_url = 'https://api.ethplorer.io/getTopTokenHolders/'+token_id+'?apiKey=EK-4L18F-Y2jC1b7-9qC3N&limit=100'
    response = requests.get(repos_url)
    if(response.status_code == 400):
        return []
    repos = json.loads(response.text)
    return repos['holders']


holder_Address = get_holders(token_address)

In [25]:
def calc_LP_distribution(holders):
    count = 0
    for holder in holders:
        if(holder['share'] < 0.01 ):
            break
        count = count +1

    LP_avg = 100 / count
    
    return LP_avg

In [26]:
lp_Avg = calc_LP_distribution(holder_Address)   #seventh feature

In [27]:
def get_creatorAddress(pair_id,token_id):
    repos_url = 'https://api.ethplorer.io/getAddressInfo/'+token_id+'?apiKey=EK-4L18F-Y2jC1b7-9qC3N'
    response = requests.get(repos_url).text
    repos = json.loads(response)

    try:
        creator_address = repos['contractInfo']['creatorAddress']
        if(creator_address == None):
          raise ValueError
    except:
         url = 'https://etherscan.io/address/'+token_id
         try:
             response = requests.get(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})
             page_soup = BeautifulSoup(response.text, "html.parser")
             Transfers_info_table_1 = str(page_soup.find("a", {"class": "hash-tag text-truncate"}))
             creator_address = re.sub('<.+?>', '', Transfers_info_table_1, 0).strip()
             if(creator_address == 'None'):
                query = mint_query_first % pair_id
                response = run_query(query)
                creator_address = response['data']['mints'][0]['to']
         except Exception as e:
              print(e)
              creator_address = 'Fail to get Creator Address'

    # if creator_address in proxy_contracts:
    #     query = mint_query_first % pair_id
    #     response = run_query(query)
    #     creator_address = response['data']['mints'][0]['to']

    return creator_address

In [28]:
def get_creator_ratio(holders,creator_address):
  for holder in holders:
    if(holder['address'] == creator_address):
      return holder['share']

  return 0

In [29]:
creater_Address = get_creatorAddress(pair_address, token_address)
creater_holdingRatio = get_creator_ratio(holder_Address, creater_Address)

In [34]:
dfin = pd.DataFrame({"active_period": active_period, "swap_count": swap_count, "burn_mean_period": burn_mean_period, 
                     "swap_mean_period": swap_mean_period, "swapIn": swapIn, "swapOut": swapOut, "lp_avg": lp_Avg, 
                     "creater_holding_ratio": creater_holdingRatio}, index = [0])

In [35]:
dfin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   active_period          1 non-null      int64  
 1   swap_count             1 non-null      int64  
 2   burn_mean_period       1 non-null      int64  
 3   swap_mean_period       1 non-null      int64  
 4   swapIn                 1 non-null      int64  
 5   swapOut                1 non-null      int64  
 6   lp_avg                 1 non-null      float64
 7   creater_holding_ratio  1 non-null      float64
dtypes: float64(2), int64(6)
memory usage: 72.0 bytes
